<a href="https://colab.research.google.com/github/raphaelrcb/HuggingFace_Workshop/blob/main/Notebooks/HuggingFace_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizagem Com HuggingFace

In [37]:
!pip install transformers datasets evaluate accelerate

In [38]:
pip install torch

In [39]:
pip install tensorflow

# Utilizando a biblioteca Transformers e a função Pipeline

## Análise de Sentimento
Através da Pipeline é possível fazer a Análise de Sentimento de um trecho de texto, dizendo se o texto é positivo ou negativo e a sua confiança. Se não especificado, o classificador carrega um modelo pré-treinado padrão (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english), e seu *tokeneizer*


In [40]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [41]:
classifier("We will be champions tonight")

[{'label': 'POSITIVE', 'score': 0.9998332262039185}]

In [42]:
classifier("They broke up")

[{'label': 'NEGATIVE', 'score': 0.9699742794036865}]

In [43]:
classifier("They broke up, now I have a chance to go out with her")

[{'label': 'POSITIVE', 'score': 0.998725950717926}]

In [44]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


## Iterando por datasets

A função pipeline() também pode iterar através dos datasets para alguma tarefa necessária. Utilizando reconhecimento de fala automátido por exemplo, é importante carregar o dataset de áudios para que seja possível utilizar no código. Carregaremos o MInDS-14
  https://huggingface.co/datasets/PolyAI/minds14

> Adicionar aspas



In [45]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

É necessário garantir que a taxa de amostragem do dataset corresponde à taxa de amostragerm do modelo em que foi treinado, no caso: model="facebook/wav2vec2-base-960h"

In [46]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

In [47]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

In [48]:
result = speech_recognizer(dataset[3:7]["audio"])
print([d["text"] for d in result])

['HOW DO I FURN A JOINA COUT', 'CAN NOW YOU HELP ME SET UP AN JOINT LEAKACCOUNT', 'HOW TO FET UP A JOINA COWT', 'ID LIKE YOU OPEN A JOINT CAMP ON TELL ME OF WHAT PRUSSUSES I THINK']


## Utilizar outro modelo e _tokenizer_ na pipeline
a pipeline consegue acomodar qualquer modelo do Hub, sendo fácil de adaptar a pipeline para outros casos de uso. Por exemplo, utilizar um modelo que poça usar outra língua.

In [49]:
model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"

Utilize <u>AutoModelForSequenceClassification</u> e <u>AutoTokenizer</u> para carregar um modelo pré-treinado e seu _tokenizer_ associado.



In [50]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [51]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo.", "Flamengo is world Champion"])

[{'label': 'Positive', 'score': 0.9551776647567749},
 {'label': 'Negative', 'score': 0.9665061235427856},
 {'label': 'Positive', 'score': 0.7399588823318481}]

Repetindo com o TensorFlow e outro modelo

In [69]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [62]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(["Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.", "I hate this", "Yo no creo que ella es tan maligna", "FLAMENGO CAMPEÃO"])

[{'label': 'Positive', 'score': 0.9818739891052246},
 {'label': 'Negative', 'score': 0.9776108860969543},
 {'label': 'Negative', 'score': 0.8707034587860107},
 {'label': 'Positive', 'score': 0.915040910243988}]

# AutoClass

Uma <u>Autoclass</u> é um atalho que automaticamente busca a arquitetura de um modelo pré-treinado dado seu nome ou caminho. Só é necessário selecionar o Autoclass apropriada para a sua tarefa e classe de pré-processamento associada. Utilizando o exemplo anterior para replicar o resultado da pipeline():


## AutoTokenizer

Um _tokenizer_ é responsável por préprocessar um texto em um _array_ de números como entradas de um modelo. Existem diversas regras que fazem parte de um processo de _tokenização_, incluindo como separar a palavra e até qual nível as palavras devem ser separadas. O mais importante é lembrar que é necessário instanciar o _tokenizer_ com o mesmo nome de modelo para garantir que está sendo usado o mesmo processo de _tokenização_ que o modelo foi pré-treinado.


 https://huggingface.co/docs/transformers/en/tokenizer_summary



In [63]:
#Carregando um tokenizer com o AutoTokenizer
from transformers import AutoTokenizer

model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
#model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [64]:
#Passando um texto para o tokenizer
encoding = tokenizer("Flamengo é o maior do mundo.")
print(encoding)

{'input_ids': [0, 231204, 393, 36, 9311, 54, 3307, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


O tokenizer pode retornar um dicionário contendo:


*   <u>input_ids</u>: representações numéricas dos tokens
*   <u>attention_masks</u>: Indica quais tokens devem ser atendidos

O tokenizer também aceita um lista de entradas, preenchendo e truncando o texto para retornart um lote com tamanho uniforme


In [65]:
#Com PyTorch
pt_batch = tokenizer(
    ["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)
print(pt_batch)

{'input_ids': tensor([[     0, 231204,    393,     36,   9311,     54,   3307,      5,      2,
              1,      1,      1,      1,      1,      1,      1],
        [     0,    438,  62040,   7359,     56,     41,     88,   5120,   8884,
          13860, 110746,   4030,     88, 231204,      5,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [79]:
# Repetindo para o modelo Com TensorFlow

from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tf_batch = tokenizer(
    ["Flamengo é o maior do mundo.", "No puedo creer que el Real Madrid haya perdido ante el Flamengo."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf",
)
print(tf_batch)


{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101, 69109,   147,   157, 14700, 10154, 12580,   119,   102,
            0,     0,     0,     0,     0,     0,     0],
       [  101, 10181, 25200, 33954, 34660, 10126, 10117, 11565, 11887,
        46733, 50758, 15100, 10117, 69109,   119,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## AutoModel

Transformers entrega uma forma simples e unificada de carregar instâncias pré-treinadas. Isso significa que carregar um <u>AutoModel</u> parece como carregar um <u>AutoTokenizer</u>. A única diferença é selecionar o  <u>AutoModel</u> correto para a tarefa. Para classificação de texto (ou de sequência), deve-se carregar o  <u>AutoModelForSequenceClassification</u>

In [80]:
from transformers import AutoModelForSequenceClassification

model_name = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [81]:
## Para passar o lote pré-processado de entradas diretamente no modelo, deve-se apenas desenpacotar o dicionário adicionando **
# com PyTorch
pt_outputs = pt_model(**pt_batch)
print(pt_outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4877,  0.0149,  3.1038],
        [ 2.6532, -0.9971, -2.0945]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


O modelo tem a saída das ativações finais no atributo `logits`. Aplicando a função softmax, se recebe as probabilidades.:

In [82]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[0.0013, 0.0435, 0.9552],
        [0.9665, 0.0251, 0.0084]], grad_fn=<SoftmaxBackward0>)


In [84]:
# Repetindo para o TensorFlow

from transformers import TFAutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

## e passando o lote préprocessado direto para o modelo. Você pode passar o tensor como é

tf_outputs = tf_model(tf_batch)
print(tf_outputs)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-0.9493156 , -1.2007748 , -0.34890935,  0.3080133 ,  1.8443937 ],
       [ 0.79998344,  0.19925563,  0.04858016, -0.4112932 , -0.36373478]],
      dtype=float32)>, hidden_states=None, attentions=None)


O modelo tem a saída das ativações finais no atributo `logits`. Aplicando a função softmax, se recebe as probabilidades.:

In [85]:
import tensorflow as tf

tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.04262926, 0.0331513 , 0.07770714, 0.14988522, 0.6966271 ],
       [0.38019112, 0.20850152, 0.17933773, 0.11322732, 0.11874233]],
      dtype=float32)>